<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250702/%E6%9C%80%E5%A4%A7%E6%B5%81%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###準備
20250702


In [ ]:
%%capture
!pip install mip
!pip install japanize-matplotlib

# 最大流問題
本講義では、
数理計画法におけるネットワークフロー問題の基本的な一つである**最大流問題 (Maximum Flow Problem)** について解説します。

## 問題の概要
最大流問題は、あるネットワークにおいて、
ある始点（ソースノード $s$）から別の終点（シンクノード $t$）へ、
各アーク（枝）が持つ容量の制約下で物資を送るとき、
送ることが可能な物資の最大量とその輸送方法を求める問題である。
このとき、物資の流れを**フロー**と呼ぶ。

ここで考えるネットワークは、以下の2つの要素（ノードとアーク）により
構成される。

**ネットワークの構成要素:**

* **ノード (Nodes、頂点):** 始点や終点、中継基地に対応する
    * **ソース (Source, $s$):** 物資を送り出す始点ノード。
    * **シンク (Sink, $t$):** 物資が送り込まれる終点ノード。
    * **中間ノード (Intermediate Nodes):** フローが通過するだけのノード。
* **アーク (Arcs、枝):** 2つのノード間を結ぶ経路を表します。各アークには以下の属性が付随します。
    * **容量 (Capacity):** そのアーク上を単位時間に流すことができるフローの上限量。例えば、パイプの太さ、道路の車線数、通信回線の帯域幅などに相当します。
    なお、枝にはフローを流すことができる向きが定められてるとする。

なお、ソースからシンクへ到達する総フロー量を最大化する際には、
* 全ての中間ノードでフロー保存則（入ってくるフローの総量と出ていくフローの総量が等しい）を満たし、かつ、
* 各アークのフローがその容量を超えないようにします。


以下の図は、ノード数が4、枝数が5のネットワークの例である。

In [ ]:
#@title ネットワークの例
# 必要なライブラリをインポート
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語フォント対応
import numpy as np # レイアウト調整用に追

# ネットワークデータ (基本例)
nodes_for_graph = ['S', 'A', 'B', 'T']
arcs_data_for_graph = [
    ('S', 'A', 10), # (始点, 終点, 容量)
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# 有向グラフを作成
G = nx.DiGraph()

# ノードを追加
G.add_nodes_from(nodes_for_graph)

# エッジ（アーク）と容量属性を追加
for u_node, v_node, cap_value in arcs_data_for_graph:
    G.add_edge(u_node, v_node, capacity=cap_value)

# ノードの配置を決定 (手動で調整するか、より適切なレイアウトアルゴリズムを選択)
# spring_layout は毎回変わるので、固定したい場合は pos を保存するか、
# shell_layout や kamada_kawai_layout などを試す。
# ここでは、より階層的な構造を意識して手動で位置を設定します。
pos = {'S': np.array([0, 0.5]), 'A': np.array([1, 1]),
       'B': np.array([1, 0]), 'T': np.array([2, 0.5])}


# グラフの描画設定
plt.figure(figsize=(8, 4))

# ノードを描画
nx.draw_networkx_nodes(G, pos, node_size=1500, node_color='lightcoral', alpha=.3, edgecolors="black", linewidths=1.5)

# エッジ（アーク）を描画
nx.draw_networkx_edges(G, pos, width=2.0, arrowstyle='-|> ', arrowsize=30, edge_color='teal',
                       connectionstyle='arc3,rad=0.1') # 少しカーブさせる

# ノードラベルを描画
nx.draw_networkx_labels(G, pos, font_size=16, font_color='black')

# エッジラベル（容量）を描画
edge_labels = {(u_node, v_node): f"{data['capacity']}" for u_node, v_node, data in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=16, font_color='darkgreen',
                            bbox=dict(facecolor='white', alpha=0.4, edgecolor='none', pad=0.1))


plt.title("最大流問題のネットワーク例 (ノードとアーク容量)", fontsize=14)
plt.axis('off') # 軸を非表示
plt.tight_layout()
plt.show()

## 実社会での応用例
* **輸送ネットワーク:** 道路網や鉄道網において、ある地点から別の地点へ輸送できる最大の物資量や車両数を評価する。
    * 例: ある都市の物流ハブから主要な配送先都市へ、現存の道路容量のもとで1日に配送できる最大のトラック台数を計算する。
* **通信ネットワーク:** コンピュータネットワークにおいて、2点間で送信可能な最大のデータ転送レート（スループット）を決定する。
* **パイプラインネットワーク:** 石油やガスのパイプライン網で、供給地から消費地へ送れる最大の流量を計算する。
* **生産システム:** 製造ラインの各工程の処理能力をアークの容量と見なし、システム全体としての最大生産量を評価する。
* **避難計画:** 災害時に、ある区域から安全な避難場所へ単位時間あたりに避難できる最大人数を推定する。

これらの問題は、システムのボトルネックを特定し、潜在的な処理能力を評価する上で非常に重要です。

## 定式化の詳細

ここでは、具体的なネットワーク例を用いて最大流問題の定式化を行います。

### 問題設定の例

ある物流ネットワークを考えます。供給拠点Sから需要拠点Tへ、中間地点A, Bを経由して物資を輸送します。各経路（アーク）には輸送容量の上限があります。

* **ノード (Nodes):** $N = \{S, A, B, T\}$
* **ソースノード (Source):** $s = S$
* **シンクノード (Sink):** $t = T$
* **アーク (Arcs) とその容量 $u_{ij}$:**
    1.  (S, A), 容量 $u_{SA} = 10$
    2.  (S, B), 容量 $u_{SB} = 8$
    3.  (A, B), 容量 $u_{AB} = 3$
    4.  (A, T), 容量 $u_{AT} = 5$
    5.  (B, T), 容量 $u_{BT} = 12$
* **アークの集合**
$$
A = \{ (S,A), (S,B), (A,B), (A,T), (B,T) \}
$$

###ネットワークの図示

ここで定義するネットワーク構造を視覚的に確認するため、
ノード間の接続関係と各アークの容量を図示する。
以下のネットーワークで、
ノードSからノードTに向けて送ることできるフローの最大を求めることを考える。


In [ ]:
#@title ネットワークの例
# 必要なライブラリをインポート
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語フォント対応
import numpy as np # レイアウト調整用に追

# ネットワークデータ (基本例)
nodes_for_graph = ['S', 'A', 'B', 'T']
arcs_data_for_graph = [
    ('S', 'A', 10), # (始点, 終点, 容量)
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# 有向グラフを作成
G = nx.DiGraph()

# ノードを追加
G.add_nodes_from(nodes_for_graph)

# エッジ（アーク）と容量属性を追加
for u_node, v_node, cap_value in arcs_data_for_graph:
    G.add_edge(u_node, v_node, capacity=cap_value)

# ノードの配置を決定 (手動で調整するか、より適切なレイアウトアルゴリズムを選択)
# spring_layout は毎回変わるので、固定したい場合は pos を保存するか、
# shell_layout や kamada_kawai_layout などを試す。
# ここでは、より階層的な構造を意識して手動で位置を設定します。
pos = {'S': np.array([0, 0.5]), 'A': np.array([1, 1]),
       'B': np.array([1, 0]), 'T': np.array([2, 0.5])}


# グラフの描画設定
plt.figure(figsize=(8, 4))

# ノードを描画
nx.draw_networkx_nodes(G, pos, node_size=1500, node_color='lightcoral', alpha=.3, edgecolors="black", linewidths=1.5)

# エッジ（アーク）を描画
nx.draw_networkx_edges(G, pos, width=2.0, arrowstyle='-|> ', arrowsize=30, edge_color='teal',
                       connectionstyle='arc3,rad=0.1') # 少しカーブさせる

# ノードラベルを描画
nx.draw_networkx_labels(G, pos, font_size=16, font_color='black')

# エッジラベル（容量）を描画
edge_labels = {(u_node, v_node): f"{data['capacity']}" for u_node, v_node, data in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=16, font_color='darkgreen',
                            bbox=dict(facecolor='white', alpha=0.4, edgecolor='none', pad=0.1))


plt.title("最大流問題のネットワーク例 (ノードとアーク容量)", fontsize=14)
plt.axis('off') # 軸を非表示
plt.tight_layout()
plt.show()

### 決定変数
最大流問題を数理モデルとして表現するために、以下の決定変数を定義します。

* $x_{ij}$: アーク $(i,j)$ 上を流れるフローの量（非負）。
* $v_{total}$: ソース $s$ からシンク $t$ への総フロー量。

### 目的関数

目的は、ソース $s$ からシンク $t$ への総フロー量 $v_{total}$ を最大化することです。

$$\text{Maximize} \quad Z = v_{total}$$

### 制約条件

1.  **フロー保存則 (Flow Conservation):**
    * **ソースノード $s$:**
ソース $s$ から出ていく正味のフロー量は
$v_{total}$ に等しい。
$$\sum_{j \text{ s.t. } (s,j) \in A} x_{sj} = v_{total}$$

    * **シンクノード $t$:**
シンク $t$ へ入ってくる正味のフロー量は
$v_{total}$ に等しい。
$$\sum_{i \text{ s.t. } (i,t) \in A} x_{it} = v_{total}
$$
    * **中間ノード $k \in N \setminus \{s, t\}$:** 各中間ノード $k$ において、入ってくる総フロー量と出ていく総フロー量は等しい。
        $$\sum_{j \text{ s.t. } (k,j) \in A} x_{kj} - \sum_{i \text{ s.t. } (i,k) \in A} x_{ik} = 0$$

2.  **容量制約 (Capacity Constraints):** 各アーク $(i,j) \in A$ 上を流れるフロー量 $x_{ij}$ は、そのアークの容量 $u_{ij}$ を超えることはできません。また、フロー量は非負である必要があります。
$$0 \le x_{ij} \le u_{ij} \quad \forall (i,j) \in A$$
3.  **総フロー量非負制約:**
$$v_{total} \ge 0$$

---
## 数理モデル（数式表記）

上記をまとめると、最大流問題の数理モデルは以下のように記述されます。

**集合とパラメータ:**
* $N$: ノードの集合
* $A$: アークの集合 (各アークは $(i,j)$ というノードのペアで、$i, j \in N$)
* $s \in N$: ソースノード
* $t \in N$: シンクノード
* $u_{ij}$: アーク $(i,j)$ の容量（上限）

**決定変数:**
* $x_{ij}$: アーク $(i,j)$ 上を流れるフローの量
* $v_{total}$: ソースからシンクへの総フロー量

**目的関数:**
$$\text{Maximize} \quad v_{total}$$

**制約条件:**
$$
\sum_{j \text{ s.t. } (s,j) \in A} x_{sj} = v_{total}
$$

$$
\sum_{i \text{ s.t. } (i,t) \in A} x_{it} = v_{total}
$$

$$
\sum_{j \text{ s.t. } (k,j) \in A} x_{kj} - \sum_{i \text{ s.t. } (i,k) \in A} x_{ik} = 0 \quad \forall k \in N \setminus \{s, t\}
$$

$$
0 \le x_{ij} \le u_{ij} \quad \forall (i,j) \in A
$$

$$
v_{total} \ge 0
$$


## Python MIP を用いた実装例

先ほどの例題を `python-mip` で最適化する。


In [ ]:
import pandas as pd
from mip import Model, xsum, maximize, CONTINUOUS, OptimizationStatus # maximize をインポートする

# 問題データ定義
nodes_mip = ['S', 'A', 'B', 'T']
source_node_mip = 'S'
sink_node_mip = 'T'

# アークの定義: (始点, 終点, 容量)
# arcs_data_for_graph と同じデータを使用
arcs_data_mip = [
    ('S', 'A', 10),
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# アークとその容量を辞書として格納
defined_arcs_mip = [(u, v) for u, v, cap in arcs_data_mip]
capacities_mip = {(u, v): cap for u, v, cap in arcs_data_mip}

# モデルの作成
model_maxflow = Model("MaxFlowProblem") # sense=mip.MAXIMIZE の設定を行わない

# 決定変数: x_flow[i,j] はアーク(i,j)上のフロー量
x_flow = {(i, j): model_maxflow.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities_mip[i,j], var_type=CONTINUOUS)
          for (i,j) in defined_arcs_mip}

# 決定変数: v_total_flow は総フロー量
v_total_flow = model_maxflow.add_var(name="total_flow", lb=0, var_type=CONTINUOUS)

# 目的関数: 総フロー量 v_total_flow の最大化
model_maxflow.objective = maximize(v_total_flow)

# 制約条件: フロー保存則
for k_node in nodes_mip:
    # ノードk_nodeから出ていくフローの総和
    flow_out = xsum(x_flow[i,j] for (i,j) in defined_arcs_mip if i == k_node)
    # ノードk_nodeに入ってくるフローの総和
    flow_in = xsum(x_flow[i,j] for (i,j) in defined_arcs_mip if j == k_node)

    if k_node == source_node_mip:
        model_maxflow += flow_out == v_total_flow, f"FlowBalance_{k_node}"
    elif k_node == sink_node_mip:
        model_maxflow += flow_in  == v_total_flow, f"FlowBalance_{k_node}"
    else: # 中間ノード
        model_maxflow += flow_out - flow_in == 0, f"FlowBalance_{k_node}"

# モデルの最適化
status = model_maxflow.optimize()

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。最大フロー量: {model_maxflow.objective_value:.2f}")
    print("各アークのフロー量:")
    flow_results_list = [] # DataFrame用のリスト
    for (i,j) in defined_arcs_mip:
        if x_flow[i,j].x > 1e-6: # ごく小さいフローは無視（表示用）
            print(f"  アーク ({i} -> {j}): {x_flow[i,j].x:.2f} ユニット (容量: {capacities_mip[i,j]})")
            flow_results_list.append({"始点": i, "終点": j, "フロー量": x_flow[i,j].x, "容量": capacities_mip[i,j]})

elif status == mip.OptimizationStatus.FEASIBLE:
    print(f"実行可能解が見つかりました（最適ではない可能性あり）。最大フロー量: {model_maxflow.objective_value:.2f}")
elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

なお、上のコードでは、モデル作成時に
```
model_maxflow = Model("MaxFlowProblem") # sense=mip.MAXIMIZE の設定を行わない
```
とする代わりに、目的関数を定める際に

```
model_maxflow.objective = maximize(v_total_flow)
```
と設定することで、最大化問題であることを指定する方法を用いている。

最小化問題の場合であれば、 `minimize()` 関数が用いられる。


In [ ]:
#@title 結果の図示
import pandas as pd

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL:
    flow_results_list = [] # DataFrame用のリスト
    for (i,j) in defined_arcs_mip:
        if x_flow[i,j].x > 1e-6: # ごく小さいフローは無視（表示用）
            flow_results_list.append({"始点": i, "終点": j, "フロー量": x_flow[i,j].x, "容量": capacities_mip[i,j]})

    if flow_results_list:
        # --- 最適フローの図示 ---
        G_sol = nx.DiGraph()
        G_sol.add_nodes_from(nodes_mip)
        active_edges_with_flow = []
        for res in flow_results_list:
            if res["フロー量"] > 1e-6:
                G_sol.add_edge(res["始点"], res["終点"], flow=res["フロー量"], capacity=res["容量"])
                active_edges_with_flow.append((res["始点"], res["終点"]))

        # pos は問題設定時のものを使用
        pos_sol = {'S': np.array([0, 0.5]), 'A': np.array([1, 1]),
                   'B': np.array([1, 0]), 'T': np.array([2, 0.5])}


        plt.figure(figsize=(8, 4))
        nx.draw_networkx_nodes(G_sol, pos_sol, node_size=1400, node_color='lightgreen', alpha=0.5, edgecolors="black", linewidths=1.5)
        nx.draw_networkx_labels(G_sol, pos_sol, font_size=16, font_weight='bold', font_color='black')

        # 全てのアークを灰色で薄く描画（背景として）
        all_edges = [(u,v) for u,v,cap in arcs_data_mip]
        nx.draw_networkx_edges(G_sol, pos_sol, edgelist=all_edges, width=1.0, alpha=0.2, edge_color='gray',
                               arrowstyle='-|> ', arrowsize=20, connectionstyle='arc3,rad=0.1')


        # フローのあるアークを太く描画
        nx.draw_networkx_edges(G_sol, pos_sol, edgelist=active_edges_with_flow, width=2.5, arrowstyle='-|> ',
                               arrowsize=25, edge_color='blue', connectionstyle='arc3,rad=0.1')

        edge_flow_labels = {(u,v): f"{d['flow']:.0f}/{d['capacity']}" for u,v,d in G_sol.edges(data=True) if d.get('flow',0)>1e-6}
        nx.draw_networkx_edge_labels(G_sol, pos_sol, edge_labels=edge_flow_labels, font_size=16, font_color='blue',
                                     bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=0.1))

        plt.title(f"最大フローの解 (総フロー: {model_maxflow.objective_value:.2f})", fontsize=12)
        plt.axis('off')
        plt.tight_layout()
        plt.show()


### 結果の解釈

上記のコードを実行すると、ソースノードSからシンクノードTへ流すことのできる最大の総フロー量が計算されます。
* **最大フロー量:** 最大化された目的関数の値 ($v_{total}$) です。これは、このネットワークがSからTに送り出せる最大量を示します。
* **各アークのフロー量:** どの経路（アーク）をどれだけの量で利用しているかを示します。フロー量がそのアークの容量上限に達しているアークは**「飽和している」**と言い、これらがネットワーク全体のボトルネックを形成する一部となります。

この結果から、ネットワークの最大輸送能力や、輸送能力向上のためにどの経路の容量を増強すべきか（ボトルネックの解消）といった洞察を得ることができます。


##最小カット問題

最大流問題と密接に関連した最適化問題に**最小カット問題**がある。

* **最小カット問題 (Min-Cut Problem)**:
ソースとシンクを分離する任意のカット（ノード集合をS側とT側に分ける境界）のうち、容量が最小となるカット（最小カット）を求める問題を**最小カット問題 (Min-Cut Problem)**と呼ぶ。

最小カットとは、
ネットワークの最も狭い部分（ボトルネック）を特定することと考えられる。


**最大流最小カット定理**によると、ソースからシンクへの最大フロー量は、容量が最小となるカット（最小カット）の容量に等しくなることが証明されてる。
証明は、この講義の範囲を超えるので扱わない。

### 例
ノード集合を、Sを含む集合とTを含む集合とに分ける方法は複数考えられる。
例えば、以下の図に示すように
* S（赤色のノード）
* A,B,T（水色のノード）

と分けた場合、
赤のノードから水色のノードに向かう枝をカットと呼ぶ。
さらに、カットとなる枝の容量の合計を**カット容量**と呼ぶ。
この時のカット容量は $10+8$ より18である。

In [ ]:
#@title 図１（S と A,B,T）
# 必要なライブラリをインポート
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語フォント対応 (pip install japanize-matplotlib が必要)
import numpy as np

# --- ネットワークデータ (最大流問題の基本例と同じ) ---
nodes_for_graph = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'
arcs_data_for_graph = [
    ('S', 'A', 10), # (始点, 終点, 容量)
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# 有向グラフを作成
G = nx.DiGraph()
G.add_nodes_from(nodes_for_graph)
for u_node, v_node, cap_value in arcs_data_for_graph:
    G.add_edge(u_node, v_node, capacity=cap_value)

# --- 最小カットの定義 ---
# 例として、N_S = {S, A}, N_T = {B, T} というカットを考えます。
# このカットの容量は、(S,B), (A,B), (A,T) の容量の合計 = 8 + 3 + 5 = 16 となります。
# これは、このネットワークの最大フロー量と一致します。
nodes_S_set = {'S'}
nodes_T_set = {'A', 'B', 'T'}

# カットを横切るアーク（N_S から N_T へ向かうアーク）を特定
cut_edges = []
cut_capacity = 0
for u_node, v_node, data in G.edges(data=True):
    if u_node in nodes_S_set and v_node in nodes_T_set:
        cut_edges.append((u_node, v_node))
        cut_capacity += data['capacity']

# ノードの配置を固定 (メイン例と同じ配置)
pos = {'S': np.array([0, 0.5]), 'A': np.array([1, 1]),
       'B': np.array([1, 0]), 'T': np.array([2, 0.5])}

# --- グラフの描画 ---
plt.figure(figsize=(5, 4))

# ノードの色分け
node_colors = []
for node in G.nodes():
    if node in nodes_S_set:
        node_colors.append('lightcoral') # N_S セットのノード色
    elif node in nodes_T_set:
        node_colors.append('skyblue') # N_T セットのノード色
    else:
        node_colors.append('lightgrey') # その他のノード（この例では存在しない）

# ノードを描画
nx.draw_networkx_nodes(G, pos, node_size=1400, node_color=node_colors, alpha=0.9, edgecolors="black", linewidths=1.5)
nx.draw_networkx_labels(G, pos, font_size=16, font_weight='bold', font_color='black')

# 全てのアークを通常スタイルで描画
nx.draw_networkx_edges(G, pos, width=1.5, arrowstyle='-|> ', arrowsize=20, edge_color='gray',
                       connectionstyle='arc3,rad=0.1')

# カットを横切るアークを強調表示
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, width=3.0, arrowstyle='-|> ', arrowsize=30,
                       edge_color='red', connectionstyle='arc3,rad=0.1', label=f"カットアーク (容量合計: {cut_capacity})")

# アークの容量ラベルを描画
edge_capacity_labels = {(u_node, v_node): f"{data['capacity']}" for u_node, v_node, data in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_capacity_labels, font_size=14, font_color='black',
                              bbox=dict(facecolor='white', alpha=0.4, edgecolor='none', pad=0.1))

plt.title(f"カットの例 (N_S={nodes_S_set}, N_T={nodes_T_set}) カット容量 = {cut_capacity}", fontsize=10)
#plt.text(0.5, -0.2, "最大流・最小カット定理: 最大フロー量 = 最小カット容量", fontsize=12, ha='center', color='darkblue')
plt.axis('off')
plt.tight_layout()
plt.show()


ノード集合を

* S, B
* A, T

と分けた場合のカットを図示すると以下のようになる。
カットはSを含む集合からTを含む集合への枝であるから、
枝（A,B）はカットには含まれない。
カット容量は10＋12より、22となる。

In [ ]:
#@title 図２（S,B と A,T）
# 必要なライブラリをインポート
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語フォント対応 (pip install japanize-matplotlib が必要)
import numpy as np

# --- ネットワークデータ (最大流問題の基本例と同じ) ---
nodes_for_graph = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'
arcs_data_for_graph = [
    ('S', 'A', 10), # (始点, 終点, 容量)
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# 有向グラフを作成
G = nx.DiGraph()
G.add_nodes_from(nodes_for_graph)
for u_node, v_node, cap_value in arcs_data_for_graph:
    G.add_edge(u_node, v_node, capacity=cap_value)

# --- 最小カットの定義 ---
# 例として、N_S = {S, A}, N_T = {B, T} というカットを考えます。
# このカットの容量は、(S,B), (A,B), (A,T) の容量の合計 = 8 + 3 + 5 = 16 となります。
# これは、このネットワークの最大フロー量と一致します。
nodes_S_set = {'S', 'B'}
nodes_T_set = {'A', 'T'}

# カットを横切るアーク（N_S から N_T へ向かうアーク）を特定
cut_edges = []
cut_capacity = 0
for u_node, v_node, data in G.edges(data=True):
    if u_node in nodes_S_set and v_node in nodes_T_set:
        cut_edges.append((u_node, v_node))
        cut_capacity += data['capacity']

# ノードの配置を固定 (メイン例と同じ配置)
pos = {'S': np.array([0, 0.5]), 'A': np.array([1, 1]),
       'B': np.array([1, 0]), 'T': np.array([2, 0.5])}

# --- グラフの描画 ---
plt.figure(figsize=(5, 4))

# ノードの色分け
node_colors = []
for node in G.nodes():
    if node in nodes_S_set:
        node_colors.append('lightcoral') # N_S セットのノード色
    elif node in nodes_T_set:
        node_colors.append('skyblue') # N_T セットのノード色
    else:
        node_colors.append('lightgrey') # その他のノード（この例では存在しない）

# ノードを描画
nx.draw_networkx_nodes(G, pos, node_size=1400, node_color=node_colors, alpha=0.9, edgecolors="black", linewidths=1.5)
nx.draw_networkx_labels(G, pos, font_size=16, font_weight='bold', font_color='black')

# 全てのアークを通常スタイルで描画
nx.draw_networkx_edges(G, pos, width=1.5, arrowstyle='-|> ', arrowsize=20, edge_color='gray',
                       connectionstyle='arc3,rad=0.1')

# カットを横切るアークを強調表示
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, width=3.0, arrowstyle='-|> ', arrowsize=30,
                       edge_color='red', connectionstyle='arc3,rad=0.1', label=f"カットアーク (容量合計: {cut_capacity})")

# アークの容量ラベルを描画
edge_capacity_labels = {(u_node, v_node): f"{data['capacity']}" for u_node, v_node, data in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_capacity_labels, font_size=14, font_color='black',
                              bbox=dict(facecolor='white', alpha=0.4, edgecolor='none', pad=0.1))

plt.title(f"カットの例 (N_S={nodes_S_set}, N_T={nodes_T_set}) カット容量 = {cut_capacity}", fontsize=10)
#plt.text(0.5, -0.2, "最大流・最小カット定理: 最大フロー量 = 最小カット容量", fontsize=12, ha='center', color='darkblue')
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
#@title 図３（S,A と B,T）
# 必要なライブラリをインポート
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語フォント対応 (pip install japanize-matplotlib が必要)
import numpy as np

# --- ネットワークデータ (最大流問題の基本例と同じ) ---
nodes_for_graph = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'
arcs_data_for_graph = [
    ('S', 'A', 10), # (始点, 終点, 容量)
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# 有向グラフを作成
G = nx.DiGraph()
G.add_nodes_from(nodes_for_graph)
for u_node, v_node, cap_value in arcs_data_for_graph:
    G.add_edge(u_node, v_node, capacity=cap_value)

# --- 最小カットの定義 ---
# 例として、N_S = {S, A}, N_T = {B, T} というカットを考えます。
# このカットの容量は、(S,B), (A,B), (A,T) の容量の合計 = 8 + 3 + 5 = 16 となります。
# これは、このネットワークの最大フロー量と一致します。
nodes_S_set = {'S', 'A'}
nodes_T_set = {'B', 'T'}

# カットを横切るアーク（N_S から N_T へ向かうアーク）を特定
cut_edges = []
cut_capacity = 0
for u_node, v_node, data in G.edges(data=True):
    if u_node in nodes_S_set and v_node in nodes_T_set:
        cut_edges.append((u_node, v_node))
        cut_capacity += data['capacity']

# ノードの配置を固定 (メイン例と同じ配置)
pos = {'S': np.array([0, 0.5]), 'A': np.array([1, 1]),
       'B': np.array([1, 0]), 'T': np.array([2, 0.5])}

# --- グラフの描画 ---
plt.figure(figsize=(5, 4))

# ノードの色分け
node_colors = []
for node in G.nodes():
    if node in nodes_S_set:
        node_colors.append('lightcoral') # N_S セットのノード色
    elif node in nodes_T_set:
        node_colors.append('skyblue') # N_T セットのノード色
    else:
        node_colors.append('lightgrey') # その他のノード（この例では存在しない）

# ノードを描画
nx.draw_networkx_nodes(G, pos, node_size=1400, node_color=node_colors, alpha=0.9, edgecolors="black", linewidths=1.5)
nx.draw_networkx_labels(G, pos, font_size=16, font_weight='bold', font_color='black')

# 全てのアークを通常スタイルで描画
nx.draw_networkx_edges(G, pos, width=1.5, arrowstyle='-|> ', arrowsize=20, edge_color='gray',
                       connectionstyle='arc3,rad=0.1')

# カットを横切るアークを強調表示
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, width=3.0, arrowstyle='-|> ', arrowsize=30,
                       edge_color='red', connectionstyle='arc3,rad=0.1', label=f"カットアーク (容量合計: {cut_capacity})")

# アークの容量ラベルを描画
edge_capacity_labels = {(u_node, v_node): f"{data['capacity']}" for u_node, v_node, data in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_capacity_labels, font_size=14, font_color='black',
                              bbox=dict(facecolor='white', alpha=0.4, edgecolor='none', pad=0.1))

plt.title(f"最小カットの例 (N_S={nodes_S_set}, N_T={nodes_T_set}) カット容量 = {cut_capacity}", fontsize=12)
#plt.text(0.5, -0.2, "最大流・最小カット定理: 最大フロー量 = 最小カット容量", fontsize=12, ha='center', color='darkblue')
plt.axis('off')
plt.tight_layout()
plt.show()


上の図のように、様々なカットが考えられるが、カット容量が**最小**となるものを見つける問題が**最小カット問題**と呼ばれ、上の例では図３で示すカットが容量が最小となる。この時のカット容量は $8+3+5$で16である。

16という値は、ノードSからノードTへの**最大流**の値16と関らず一致することが知られている。（証明はこの講義の範囲を超えるので行わない。）

##その他モデルの拡張

* **複数ソース・複数シンク:** 問題が複数のソースやシンクを持つ場合、**スーパーソース**と**スーパーシンク**というダミーノードを導入することで、標準的な単一ソース・単一シンクの最大流問題に変換できます。スーパーソースから各実際のソースへ、各実際のシンクからスーパーシンクへアークを追加します。
* **ノード容量:** アークだけでなくノード自体に通過できるフローの上限（ノード容量）がある場合。これは、各ノードを2つのノード（入力側ノード、出力側ノード）に分割し、その間に容量制限を持つアークを設けることでモデル化できます。
* **費用を伴うフロー:** 最大フローを達成しつつ、費用も考慮したい場合は、最小費用流問題や、より複雑な最小費用最大流問題（最大フローを達成するフローの中で費用最小のものを見つけるなど）を検討する必要があります。


---
#演習問題

##演習問題1

**問題:**
例題のアーク (A, T) の容量を 5 から 7 増やし、
アーク (S, B) の容量を 8 から 6 に減らした場合
最大フロー量を求めよ。


##演習問題2

**問題:**
例題のネットワークに、新たにノードBからノードAへのアーク (B, A) を追加することを考えます。この新しいアークの容量は 2 であるとします。
この時の最大フロー量を求めよ。


## 演習問題3

**問題:**
例題のネットワークにおいて、アーク (A, B) が使用できなくなったとします。
この時、最大フローを求めよ。


## 演習問題4

**問題:**
例題のネットワークでは、アーク (S,B) 容量8、(A,B) 容量3、(A,T) 容量5 が飽和していました（フローが容量上限に達している）。

1.  飽和していないアーク (S,A)（フロー8、容量10）の容量を10から12に増やした場合、最大フロー量を求めよ。

2.  次に、元の容量に戻した後、飽和しているアーク (A,T) の容量を5から7に増やした場合、最大フロー量を求めよ。


##演習問題5:

**問題:**
以下の複数ソース・複数シンクを持つネットワークを考える。
* ソース: S1, S2
* シンク: T1, T2
* 中間ノード: M
* アークと容量:
    * (S1, M, 10), (S2, M, 8)
    * (M, T1, 7), (M, T2, 9)

このネットワークでのソースノードからシンクノードへの最大フロー量を求めよ。


In [ ]:
#@title 複数ソース、複数シンクのグラフ
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語フォント対応
import numpy as np

# 元の複数ソース・複数シンクネットワークのデータ
original_nodes_msms = ['S1', 'S2', 'M', 'T1', 'T2']
original_arcs_data_msms = [
    ('S1', 'M', 10), ('S2', 'M', 8),
    ('M', 'T1', 7), ('M', 'T2', 9)
]

# 有向グラフを作成
G_original_msms = nx.DiGraph()
G_original_msms.add_nodes_from(original_nodes_msms)
for u, v, cap in original_arcs_data_msms:
    G_original_msms.add_edge(u, v, capacity=cap)

# ノードの配置を階層的に設定
pos_original_msms = {
    'S1': np.array([0, 1]), 'S2': np.array([0, 0]),    # ソース群
    'M':  np.array([1, 0.5]),                          # 中間ノード
    'T1': np.array([2, 1]), 'T2': np.array([2, 0])     # シンク群
}

# グラフの描画設定
plt.figure(figsize=(4.5, 3.5))
node_draw_size_orig = 1400

# ノードの色分け
node_colors_original = []
for node in G_original_msms.nodes():
    if 'S' in node:
        node_colors_original.append('lightcoral') # 元のソース
    elif 'T' in node:
        node_colors_original.append('skyblue')    # 元のシンク
    else:
        node_colors_original.append('lightgrey')  # 中間ノード

nx.draw_networkx_nodes(G_original_msms, pos_original_msms, node_size=node_draw_size_orig,
                       node_color=node_colors_original, alpha=0.9, edgecolors="black", linewidths=1.5)
nx.draw_networkx_labels(G_original_msms, pos_original_msms, font_size=14, font_weight='bold', font_color='black')

nx.draw_networkx_edges(
    G_original_msms,
    pos_original_msms,
    width=1.8,
    arrowstyle='-|> ',
    arrowsize=20,
    edge_color='teal',
    connectionstyle='arc3,rad=0.1',
    node_size=node_draw_size_orig,
    min_source_margin=15,
    min_target_margin=15
)

edge_labels_original = {(u,v): f"{d['capacity']}" for u,v,d in G_original_msms.edges(data=True)}
nx.draw_networkx_edge_labels(
    G_original_msms,
    pos_original_msms,
    edge_labels=edge_labels_original,
    font_size=16,
    font_color='darkgreen',
    bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=0.1)
)

plt.title("複数ソース・複数シンクネットワーク", fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
#@title スーパーソースとスーパーシンクを追加した図
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語フォント対応
import numpy as np

# 変換後のネットワークデータ
nodes_transformed = ['SS', 'S1', 'S2', 'M', 'T1', 'T2', 'ST']
arcs_data_transformed = [
    ('SS', 'S1', 34), ('SS', 'S2', 34),  # スーパーソースからのアーク
    ('S1', 'M', 10), ('S2', 'M', 8),   # 元のネットワークのアーク
    ('M', 'T1', 7),  ('M', 'T2', 9),    # 元のネットワークのアーク
    ('T1', 'ST', 34), ('T2', 'ST', 34)   # 元のシンクからスーパーシンクへのアーク
]

# 有向グラフを作成
G_transformed = nx.DiGraph()
G_transformed.add_nodes_from(nodes_transformed)
for u, v, cap in arcs_data_transformed:
    G_transformed.add_edge(u, v, capacity=cap)

# ノードの配置を階層的に設定
pos_transformed = {
    'SS': np.array([0, 0.5]),
    'S1': np.array([1, 1]), 'S2': np.array([1, 0]),
    'M':  np.array([2, 0.5]),
    'T1': np.array([3, 1]), 'T2': np.array([3, 0]),
    'ST': np.array([4, 0.5])
}

# グラフの描画設定
plt.figure(figsize=(12, 5)) # 少し縦長に調整

# ノードの描画パラメータ
node_draw_size = 2200 # ノードの描画サイズ (以前より少し小さくする可能性)

# ノードの色分け
node_colors_transformed = []
for node in G_transformed.nodes():
    if node == 'SS' or node == 'ST':
        node_colors_transformed.append('gold') # スーパーノード
    elif 'S' in node:
        node_colors_transformed.append('lightcoral') # 元のソース
    elif 'T' in node:
        node_colors_transformed.append('skyblue') # 元のシンク
    else:
        node_colors_transformed.append('lightgrey') # 中間ノード

nx.draw_networkx_nodes(G_transformed, pos_transformed, node_size=node_draw_size,
                       node_color=node_colors_transformed, alpha=0.9, edgecolors="black", linewidths=1.5)
nx.draw_networkx_labels(G_transformed, pos_transformed, font_size=14, font_weight='bold', font_color='black')

# エッジ（アーク）を描画 ★改善点
nx.draw_networkx_edges(
    G_transformed,
    pos_transformed,
    width=1.8,         # 線の太さ
    arrowstyle='-|> ', # 矢印のスタイル (例: シンプルな三角形の先端)
    arrowsize=20,      # 矢印の先のサイズ
    edge_color='teal',
    connectionstyle='arc3,rad=0.1', # 線を少しカーブさせる
    node_size=node_draw_size,      # ★ノードの描画サイズをエッジ描画関数に伝える
    min_source_margin=15, # ★ソースノードからのマージン (ポイント単位)
    min_target_margin=15  # ★ターゲットノードへのマージン (ポイント単位)
)

edge_labels_transformed = {(u,v): f"{d['capacity']}" for u,v,d in G_transformed.edges(data=True)}
nx.draw_networkx_edge_labels(
    G_transformed,
    pos_transformed,
    edge_labels=edge_labels_transformed,
    font_size=16,
    font_color='darkgreen',
    bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=0.1)
)

plt.title("複数ソース・複数シンク問題から変換されたネットワーク", fontsize=14)
plt.text(pos_transformed['SS'][0]-0.35, pos_transformed['SS'][1], 'スーパー\nソース', ha='right', va='center', fontsize=12, color='black')
plt.text(pos_transformed['ST'][0]+0.35, pos_transformed['ST'][1], 'スーパー\nシンク', ha='left', va='center', fontsize=12, color='black')
plt.axis('off')
plt.tight_layout()
plt.show()